In [1]:
import pandas as pd
import numpy as np

import pickle
import joblib
import csv

from tqdm.notebook import tqdm, trange
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

from rdkit import Chem
from rdkit.Chem import AllChem

## Load Dataset

In [2]:
data_path = 'dataset/test_data_fix_concat.csv'
data_df = pd.read_csv(data_path)

In [3]:
data_df.head()

,oil_type,blend_id,oil_property_param_title,oil_property_param_value,component_name,component_class,polymer,component_property_param_title,component_property_param_value,smiles
0,c25411c1-0bec-41f3-8eee-81daaed2b890,d272c9a6-3332-11ed-9685-005056921581,7c8a81df-b7e7-4507-aab1-79a61fce7887,4.94,b26322a8-b4fa-41cc-a755-757b41d22919,Group 3 base oil,no,5cf8e492-dfea-4ecb-8799-a97989c46256,0.0000,CCC(C)CCCC
1,c25411c1-0bec-41f3-8eee-81daaed2b890,d272c9a6-3332-11ed-9685-005056921581,7c8a81df-b7e7-4507-aab1-79a61fce7887,4.94,b26322a8-b4fa-41cc-a755-757b41d22919,Group 3 base oil,no,9e2de643-ddca-487e-b9e7-69b25d8662bf,0.0000,CCC(C)CCCC
2,c25411c1-0bec-41f3-8eee-81daaed2b890,d272c9a6-3332-11ed-9685-005056921581,7c8a81df-b7e7-4507-aab1-79a61fce7887,4.94,b26322a8-b4fa-41cc-a755-757b41d22919,Group 3 base oil,no,f6cc205c-a44d-40e5-9462-2eab4a673e2a,0.0001,CCC(C)CCCC
3,c25411c1-0bec-41f3-8eee-81daaed2b890,d272c9a6-3332-11ed-9685-005056921581,7c8a81df-b7e7-4507-aab1-79a61fce7887,4.94,b26322a8-b4fa-41cc-a755-757b41d22919,Group 3 base oil,no,d6eff30f-788e-48a2-b2bd-cbba188f4e63,0.0006,CCC(C)CCCC
4,c25411c1-0bec-41f3-8eee-81daaed2b890,d272c9a6-3332-11ed-9685-005056921581,7c8a81df-b7e7-4507-aab1-79a61fce7887,4.94,b26322a8-b4fa-41cc-a755-757b41d22919,Group 3 base oil,no,049e5cb9-9c20-438c-9ef2-96870712a0be,0.0001,CCC(C)CCCC


## Remove Dublicates

In [4]:
# For oil properties that have multiple different values (for one blend), we use the median value

count_unique = data_df[np.isfinite(data_df.oil_property_param_value)].groupby(['blend_id', 'oil_property_param_title']).oil_property_param_value.unique().map(len)
multiindex = count_unique[count_unique>1].index

for idx in tqdm(multiindex):
    vals = data_df[np.isfinite(data_df.oil_property_param_value) & (data_df.blend_id == idx[0]) & (data_df.oil_property_param_title == idx[1])].oil_property_param_value
    newval = np.median(vals)
    data_df.loc[(data_df.blend_id == idx[0]) & (data_df.oil_property_param_title == idx[1]) & (data_df.oil_property_param_value != newval), 'oil_property_param_value'] = newval  

data_df.drop_duplicates(inplace=True)
len(data_df)

  0%|          | 0/10 [00:00<?, ?it/s]

109337

In [5]:
# For component properties that have multiple different values (for one blend), we use the median value

count_unique = data_df[np.isfinite(data_df.component_property_param_value)].groupby(['blend_id', 'oil_property_param_title', 'component_name', 'component_property_param_title']).component_property_param_value.unique().map(len)
multiindex = count_unique[count_unique>1].index

for idx in tqdm(multiindex):
    vals = data_df[np.isfinite(data_df.component_property_param_value) & (data_df.blend_id == idx[0]) & (data_df.oil_property_param_title == idx[1])
                        & (data_df.component_name == idx[2]) & (data_df.component_property_param_title == idx[3])].component_property_param_value
    newval = np.median(vals)
    data_df.loc[(data_df.blend_id == idx[0]) & (data_df.oil_property_param_title == idx[1])
                        & (data_df.component_name == idx[2]) & (data_df.component_property_param_title == idx[3]) 
                        & (data_df.component_property_param_value != newval), 'component_property_param_value'] = newval

data_df.drop_duplicates(inplace=True)
len(data_df)

  0%|          | 0/241 [00:00<?, ?it/s]

109096

In [6]:
# For component properties that have both a value and NaN (for one blend), we take the non-NaN value

count_unique = data_df.groupby(['blend_id', 'oil_property_param_title', 'component_name', 'component_property_param_title']).component_property_param_value.unique().map(len)
multiindex = count_unique[count_unique>1].index

for idx in tqdm(multiindex):
    newval = data_df[np.isfinite(data_df.component_property_param_value) & (data_df.blend_id == idx[0]) & (data_df.oil_property_param_title == idx[1])
                        & (data_df.component_name == idx[2]) & (data_df.component_property_param_title == idx[3])].component_property_param_value.mean()
    
    data_df.loc[(data_df.blend_id == idx[0]) & (data_df.oil_property_param_title == idx[1])
                        & (data_df.component_name == idx[2]) & (data_df.component_property_param_title == idx[3]) 
                        & (data_df.component_property_param_value != newval), 'component_property_param_value'] = newval

data_df.drop_duplicates(inplace=True)
len(data_df)

  0%|          | 0/1781 [00:00<?, ?it/s]

107315

## Load Model

In [7]:
model = joblib.load('model/model.joblib') 

In [8]:
comp_props = pickle.load(open('model/comp_props', 'rb'))
oil_props = pickle.load(open('model/oil_props', 'rb'))

mean_compprops = pickle.load(open('model/mean_compprops', 'rb'))
mean_oilprops = pickle.load(open('model/mean_oilprops', 'rb'))

oiltype2vec = pickle.load(open('model/oiltype2vec', 'rb'))
comptype2num = pickle.load(open('model/comptype2num', 'rb'))

In [9]:
base_embed = np.load('model/base_embed.npy')

## Feature Engineering

### SMILES Embeddings Generation

In [11]:
unique_smiles = list(data_df.smiles[~data_df.smiles.isnull()].unique())
print(f'Number of unique SMILES: {len(unique_smiles)}')

Number of unique SMILES: 54


In [12]:
elements = ['O', 'H', 'Cl', 'C']

In [13]:
embeddings = []

for smiles in tqdm(unique_smiles):    
    mol = Chem.MolFromSmiles(smiles)
    mol_h = Chem.AddHs(mol)

    num_atoms = mol_h.GetNumAtoms()
    num_heavy = mol_h.GetNumHeavyAtoms()
    num_atomatic = len(mol_h.GetAromaticAtoms())
    mol_elements = list(map(lambda x: x.GetSymbol(), mol_h.GetAtoms()))

    x = [num_atoms, num_heavy, num_atomatic]
    for element in elements:
        if element in mol_elements:
            x.append(mol_elements.count(element))
        else:
            x.append(0)

    AllChem.EmbedMolecule(mol_h, useRandomCoords=True)
    try:
        AllChem.MMFFOptimizeMolecule(mol_h, maxIters=10000)
    except ValueError:
        pass

    num_acceptors = Chem.rdMolDescriptors.CalcNumHBA(mol_h)
    num_donors = Chem.rdMolDescriptors.CalcNumHBD(mol_h)
    tpsa = Chem.rdMolDescriptors.CalcTPSA(mol_h)
    vol = AllChem.ComputeMolVolume(mol_h,gridSpacing=0.1)
    x.append(num_acceptors)
    x.append(num_donors)
    x.append(tpsa)
    x.append(vol)
    
    embeddings.append(x)

embeddings = np.array(embeddings)

  0%|          | 0/54 [00:00<?, ?it/s]

In [14]:
smiles2embedding = {unique_smiles[i] : embeddings[i] for i in range(len(unique_smiles))}

### Feature Construction

In [15]:
blend_ids = []
Xs = []

for blend_id in tqdm(list(data_df.blend_id.unique())):
    blend_ids.append(blend_id)
    
    feature_vector = []

    # oil type (one-hot) -> 4-dimensional vector
    oiltype_list = data_df[data_df.blend_id == blend_id].oil_type.unique()
    assert len(oiltype_list) == 1, 'Several oil types for one blend'
    oil_type = oiltype_list[0]
    
    try: oiltype_vec = oiltype2vec[oil_type]
    except: oiltype_vec = [1/4]*4
    feature_vector += oiltype_vec

    # oil props -> 15-dimensional vector
    oilprops_vec = []
    assert max(data_df[data_df.blend_id == blend_id].groupby('oil_property_param_title').oil_property_param_value.unique().apply(lambda x: len(x)).values) == 1
    oilprop2val = data_df[(data_df.blend_id == blend_id) & np.isfinite(data_df.oil_property_param_value)].groupby('oil_property_param_title').oil_property_param_value.mean().to_dict()
    
    for oil_prop in oil_props:
        try: oilprops_vec += [oilprop2val.get(oil_prop, mean_oilprops[(oil_prop, oil_type)])]
        except: oilprops_vec += [oilprop2val.get(oil_prop, mean_oilprops[(oil_prop, '')])]
    feature_vector += oilprops_vec
    
    # component types (bag of words) -> 13-dimensional vector
    comp_type_bow = [0]*13
    blend_comp_types = list(data_df[data_df.blend_id == blend_id].groupby('component_name').component_class.min().values)
    
    for comp_type in blend_comp_types:
        try: comp_type_bow[comptype2num[comp_type]] += 1
        except:
            for i in range(13):
                comp_type_bow[i] += 1/13
    feature_vector += comp_type_bow

    # component properties -> 23-dimensional vector
    comp_val_list = []
    blend_comp_names = data_df[(data_df.blend_id == blend_id)].groupby(['component_name']).component_name.min().values
    for comp_name in blend_comp_names:
        compprop2val = data_df[(data_df.blend_id == blend_id) & (data_df.component_name == comp_name) & np.isfinite(data_df.component_property_param_value)].groupby('component_property_param_title').component_property_param_value.mean().to_dict()
        tmp = data_df[(data_df.blend_id == blend_id) & (data_df.component_name == comp_name)].polymer.unique()
        assert len(tmp) == 1
        if isinstance(tmp[0], str): polymer = tmp[0]
        else: polymer = ''
        
        comp_val_list.append([])
        for comp_prop in comp_props:
            comp_val_list[-1].append(compprop2val.get(comp_prop, mean_compprops[(comp_prop, polymer)]))
    feature_vector += list(np.array(comp_val_list).mean(axis=0))
    
    # smiles rdkit embeds -> 11-dimensional vector
    smiles_embed_list = []
    smiles_num = (~data_df[(data_df.blend_id == blend_id)].groupby('component_name').smiles.min().isnull()).sum()
    if smiles_num == 0:
        smiles_embed_list = [base_embed]
    else:
        smiles_list = list(data_df[(data_df.blend_id == blend_id)].groupby('component_name').smiles.min().values)
        for smiles in smiles_list:
            if isinstance(smiles, str):
                smiles_embed_list.append(smiles2embedding[smiles])
    feature_vector += list(np.array(smiles_embed_list).mean(axis=0))
    
    Xs.append(feature_vector)

Xs = np.array(Xs)

  0%|          | 0/138 [00:00<?, ?it/s]

In [16]:
print(Xs.shape)

(138, 66)


## Predict

In [17]:
ys_pred = np.exp(model.predict(Xs))

## Evaluate

In [18]:
data_results_path = 'dataset/test_set_private.csv'
data_results_df = pd.read_csv(data_results_path)
real_values_dict = data_results_df.set_index('blend_id')['oil_property_param_value'].to_dict()

In [19]:
ys = [real_values_dict[blend_id] for blend_id in blend_ids]
ys = np.array(ys)

In [20]:
print(f'MAE:{np.abs(ys-ys_pred).mean()/1e3:.1f}, RMSD:{np.sqrt(np.square(ys-ys_pred).mean())/1e3:.1f}')

MAE:19.1, RMSD:47.6


In [21]:
coefficient_of_dermination = r2_score(np.log(ys), np.log(ys_pred))
print(f'{coefficient_of_dermination:.3f}')

0.839


In [22]:
with open('results/test_preds.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['blend_id', 'y_real', 'y_pred'])
    for blend_id, y_real, y_pred in zip(blend_ids, ys, ys_pred):
        writer.writerow([blend_id, y_real, y_pred])